In [1]:
import mediapipe as mp
import cv2
import numpy as np

# Load the MediaPipe Hands model
mp_hands = mp.solutions.hands.Hands()

def get_landmarks(file_location):
    # Read the image
    image = cv2.imread(file_location)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and get the landmarks data
    results = mp_hands.process(image_rgb)

    # Extract the landmarks from the results
    landmarks = results.multi_hand_landmarks
    
    # Convert the landmarks to a numpy array
    if landmarks:
        landmarks_np = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks[0].landmark])
    else:
        return None
    
    return landmarks_np

I0000 00:00:1719362533.136455 9719168 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719362533.151127 9719552 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719362533.158115 9719552 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
get_landmarks("../data/ASL_Alphabet_Dataset/asl_alphabet_train/A/1.jpg")

/Users/gvssriram/anaconda3/envs/math_note_x/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


array([[ 4.64912236e-01,  7.08312333e-01, -5.07552727e-07],
       [ 5.55222034e-01,  6.61325216e-01, -3.01212054e-02],
       [ 6.11243725e-01,  5.56986272e-01, -3.71273123e-02],
       [ 6.28573656e-01,  4.74767298e-01, -4.52330187e-02],
       [ 6.57870531e-01,  4.19055343e-01, -4.70682345e-02],
       [ 5.59369504e-01,  4.78744626e-01, -9.29429382e-03],
       [ 5.80612838e-01,  4.29308653e-01, -5.91461286e-02],
       [ 5.69544733e-01,  4.97442067e-01, -8.42664838e-02],
       [ 5.57560682e-01,  5.53520024e-01, -9.04561877e-02],
       [ 4.97235030e-01,  4.80179012e-01, -9.58103687e-03],
       [ 5.20756185e-01,  4.38761264e-01, -7.03197271e-02],
       [ 5.21485209e-01,  5.23391068e-01, -8.48039761e-02],
       [ 5.23057640e-01,  5.91304541e-01, -7.59945437e-02],
       [ 4.39731032e-01,  4.90437806e-01, -1.82091109e-02],
       [ 4.67841029e-01,  4.89203215e-01, -7.90640488e-02],
       [ 4.80144352e-01,  5.77234149e-01, -7.65616521e-02],
       [ 4.88350570e-01,  6.36587143e-01

In [6]:
import cv2
import numpy as np

def plot_landmarks(image_path, landmarks):
    # Read the image
    image = cv2.imread(image_path)

    # Iterate over the landmarks and plot them on the image
    for landmark in landmarks:
        x, y, z = landmark
        # Convert the 3D coordinates to 2D image coordinates
        x = int(x * image.shape[1])
        y = int(y * image.shape[0])
        # Plot a circle at the landmark position
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)

    # Display the image with landmarks
    cv2.imshow("Landmarks", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
landmarks = get_landmarks("../data/ASL_Alphabet_Dataset/asl_alphabet_train/A/1.jpg")
plot_landmarks("../data/ASL_Alphabet_Dataset/asl_alphabet_train/A/1.jpg", landmarks)

/Users/gvssriram/anaconda3/envs/math_note_x/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
# Release the MediaPipe Hands model
mp_hands.close()